In [79]:
WORKSPACE_PATH = 'workspace'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'

In [80]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [81]:
import json

def json_read(data_path):
    with open(data_path, 'r') as data:
        data = json.load(data)
    return data

In [82]:
JSON_NAME = '/instances_default.json'
DATA_PATH = ANNOTATION_PATH+'/original'+ JSON_NAME
data = json_read(DATA_PATH)

Bennene Bilder im Verzeichnis images und in der Annotationsdatei um

In [83]:
import os

def rename_images(IMAGE_PATH):
    index=1
    img  = os.listdir(IMAGE_PATH)
    for i in img:
        new_name = f"{index:012d}"+'.jpeg'
        old_path = os.path.join(IMAGE_PATH, i)
        new_path = os.path.join(IMAGE_PATH, new_name)
        os.rename(old_path, new_path)
        index += 1

def rename_images_in_annotation():
    index=1
    for img in data['images']:
        new_name = f"{index:012d}"+'.jpeg'
        img['file_name'] = new_name 
        index += 1

rename_images(IMAGE_PATH+'/original')
rename_images_in_annotation()

with open(ANNOTATION_PATH+'/original' + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)  

In [84]:
images = []
bounding_boxes = []
labels = []
image_height = 1920
image_width = 1080

for image in data['images']:
    one_image = []
    one_image.append(image['id'])
    one_image.append(image['file_name'])
    images.append(one_image)

for annotation in data['annotations']:
    one_box = []
    box_and_id = []
    one_box.append(annotation['bbox'])
    bounding_boxes.append(one_box)

print(images)
print(bounding_boxes)



[[1, '000000000001.jpeg'], [2, '000000000002.jpeg'], [3, '000000000003.jpeg'], [4, '000000000004.jpeg'], [5, '000000000005.jpeg'], [6, '000000000006.jpeg'], [7, '000000000007.jpeg'], [8, '000000000008.jpeg'], [9, '000000000009.jpeg'], [10, '000000000010.jpeg'], [11, '000000000011.jpeg'], [12, '000000000012.jpeg'], [13, '000000000013.jpeg'], [14, '000000000014.jpeg'], [15, '000000000015.jpeg'], [16, '000000000016.jpeg'], [17, '000000000017.jpeg']]
[[[39.37, 366.56, 481.39, 304.52]], [[45.75, 429.5, 459.55, 41.3]], [[44.68, 472.77, 459.55, 41.3]], [[43.86, 518.93, 459.54, 41.3]], [[30.05, 669.42, 492.05, 296.7]], [[45.19, 703.01, 458.15, 44.2]], [[42.64, 749.76, 226.95, 41.65]], [[273.84, 751.46, 231.2, 39.95]], [[45.19, 794.81, 223.55, 40.8]], [[277.24, 811.81, 19.55, 22.95]], [[356.29, 812.66, 19.55, 22.95]], [[427.69, 811.81, 20.4, 22.95]], [[45.19, 839.01, 128.35, 44.2]], [[172.69, 838.16, 96.9, 46.75]], [[379.24, 842.41, 125.8, 43.35]], [[43.49, 892.56, 227.8, 41.65]], [[276.39, 890

In [85]:
def resize_imgaes(input_path, output_path, height, width):
    image =  cv2.imread(input_path)
    resized_image = cv2.resize(image, (height, width))
    cv2.imwrite(output_path, resized_image)    

def resize_bounding_boxes_and_area(input_path,bounding_boxes, height, width):
    image =  cv2.imread(input_path)
    height_ratio = height / image.shape[0]
    width_ratio = width / image.shape[1]
    
    for box in bounding_boxes:
        resized_boxes = []
        resized_area = 0

        x = np.round(box[0]*width_ratio,2)
        y = np.round(box[1]*height_ratio,2)
        x_width = np.round(box[2]*width_ratio,2)
        y_height = np.round(box[3]*height_ratio,2)
        resized_boxes.append([x, y, x_width, y_height])

        resized_area = x_width * y_height

    return x, y, x_width, y_height, resized_area


In [86]:
import os

new_height = 224
new_width = 224
OUTPUT_PATH_IMAGES = IMAGE_PATH + '/' + str(new_height) + 'x' + str(new_width)
OUTPUTPATH_ANNOTATIONS = ANNOTATION_PATH + '/' + str(new_height) + 'x' + str(new_width)

os.makedirs(OUTPUT_PATH_IMAGES, exist_ok=True)
os.makedirs(OUTPUTPATH_ANNOTATIONS, exist_ok=True)

#Copy JSON Doc in new ANNOTATIONPATH
with open(DATA_PATH, 'r') as source_file:
    data = json.load(source_file)
with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    


In [87]:
#Resize Images
for img in range(len(images)):
    resize_imgaes(IMAGE_PATH+'/original'+ '/' +images[img][1], OUTPUT_PATH_IMAGES+ '/' +images[img][1], new_height, new_width)


with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'r') as json_data:
    data = json.load(json_data)


#Resize BBox and area
for i, annotation in enumerate(data['annotations']):
    calculate_resize = resize_bounding_boxes_and_area(IMAGE_PATH+'/original'+ '/' +images[0][1],bounding_boxes[i], new_height, new_width)
    new_box = calculate_resize[0],calculate_resize[1],calculate_resize[2],calculate_resize[3]
    new_area = calculate_resize[4]
    annotation['bbox'] = new_box
    annotation['area'] = new_area

#Change height/width
for image in data['images']:
    image['height']=new_height
    image['width']=new_width

with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    

Create Dataset

In [88]:
import json

images = data['images']
annotations = data['annotations']
categories = data['categories']

dataset = []

for annotation in annotations:
    image_info = next(image for image in images if image['id'] == annotation['image_id'])
    image_path = OUTPUT_PATH_IMAGES + image_info['file_name']
    bbox = annotation['bbox']  # [x, y, width, height]
    class_id = annotation['category_id']

    xmin, ymin, width, height = bbox
    xmax, ymax = xmin + width, ymin + height

    dataset.append({
        'image_path': image_path,
        'bbox': [xmin, ymin, xmax, ymax],
        'class_id': class_id
    })

print(len(dataset))
print(dataset[0])


698
{'image_path': 'workspace/images/224x224000000000001.jpeg', 'bbox': [11.48, 60.11, 151.89, 110.05], 'class_id': 1}


In [89]:
import random
from sklearn.model_selection import train_test_split

train_temp, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, val = train_test_split(train_temp, test_size=0.2, random_state=42)

print(f"Anzahl Trainingsdaten: {len(train)}")
print(f"Anzahl Validierungsdaten: {len(val)}")
print(f"Anzahl Testdaten: {len(test)}")


Anzahl Trainingsdaten: 446
Anzahl Validierungsdaten: 112
Anzahl Testdaten: 140


In [91]:
base_model = keras_cv.models.MobileNetV3Backbone.from_preset(
    "mobilenet_v3_large_imagenet",
    load_weights=False,
)
#Freeze base model
base_model.trainable = False

base_model.summary()


Model: "mobile_net_v3_backbone_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_24 (InputLayer)       [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling_20 (Rescaling)    (None, None, None, 3)        0         ['input_24[0][0]']            
                                                                                                  
 Conv (Conv2D)               (None, None, None, 16)       432       ['rescaling_20[0][0]']        
                                                                                                  
 Conv_BatchNorm (BatchNorma  (None, None, None, 16)       64        ['Conv[0][0]']                
 lization)                                                                

New Top Layer

In [95]:

import keras
inputs = keras.Input(shape=(new_height, new_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)